# Multiprocessing!

## Process Investigation

In this first exercise, we'll play with `multiprocessing`'s `Pool.map()` to see how it functions. 

First, we create an instance of the `Pool` object and define the number of processes we want to use.
It's best practice to explicitly set this number. By default, `Pool` will set the number of processes
to the number of available cores. On a shared system like the HPCC, this can easily mean your program
will interfere with other users! We can use the context manager keyword `with` to ensure our processes
are properly ended after use.

The `Pool.map()` method applies a function to an iterable object. For this example, our function
returns some identifying information about the process running it such as its process ID and
name. The process ID is how the operating system identifies our process, while the name is internal
to Python. Our function will also return the data that was handed to the process, so we can see how `map()`
divides the work.

Run the following code block and then answer the exercises.

In [1]:
import multiprocessing

def print_info(data):
    
    this_process = multiprocessing.current_process()
    
    info = {"proc_id" : this_process.pid,
            "proc_name" : this_process.name,
            "proc_data" : data}
    
    return info

with multiprocessing.Pool(processes = 4) as p:
    # Pool.map accepts a function to apply
    # and an iterable object of data to share
    all_info = p.map(func = print_info, iterable = range(10))

---
### Exercises

1. What kind of object is `all_info`? That is, how does `Pool.map()` collect the values returned by each process?

*Answer here*

2. How many distinct processes are there in `all_info`? Does this align with expectations?

*Answer here*

3. How is the data distributed? Does each processes receive their allocated data all at once (as e.g. a list) or one at a time? Does each process have an equal amount of data?

*Answer here*

---
## Implementing Pool

In order to use `Pool.map()` we need to do two things:

1. Identify the set of objects we wish to distribute
2. Encapsulate our desired operation in a function

The set of objects we are distributing to each process can be very concrete (e.g., multiple files with data that needs processing) or very abstract. For the following walkthrough, we'll be distributing something more abstract - runs of a Monte Carlo algorithm. Broadly speaking, Monte Carlo algorithms rely on random sampling to generate a result. We'll be using a Monte Carlo method to approximate the value of $\pi$.

### About the Algorithm

Imagine we have a square with sides of length $2r$. Inside this square we can draw a circle with radius $r$. Since the area of the square is $(2r)^2$ and the area of the circle is $\pi r^2$, the ratio of the two gives us $\pi$:

$$
\frac{Area\ of\ Circle}{Area\ of\ Square} = \frac{\pi r^2}{4r^2} = \frac{\pi}{4}
$$

We can estimate the area of these two shapes by randomly selecting points, say within $-1 \le x \le 1$ and $-1 \le y \le 1$. By construction, all of the points will fall inside a square with $r=1$, so the area of the square is approximately the total number of points we have sampled. Then, we can count the number of points that fall inside the circle $x^2 + y^2 \le 1$. This number is approximately the area of the circle. Therefore,

$$
\frac{Number\ of\ Points\ in\ Circle}{Total\ Number\ of\ Points} \approx \frac{\pi}{4}
$$

An example of this procedure is shown below [[source](https://miro.medium.com/v2/resize:fit:960/1*N8AWxYj3s2WrNrddinkcvQ.png)]. Points inside the circle are colored red, while points outside the circle (but still inside the square) are dark blue.

<div>
<img src="monte_carlo_pi.png" alt="Demonstration of the Monte Carlo method for estimating pi" width="70%">
</div>

To achieve a more accurate estimate of $\pi$, we need to increase the number of points we sample. One tweak we can make to improve the efficiency of our sampling is to **only sample the upper right quadrant** ($0 \le x \le 1$ and $0 \le y \le 1$). The math to approximate $\pi$ remains the same, but our random points now more efficiently cover the area of the quadrant.

It is this variation that is implemented below. The following exercises will walk you through the process of adapting this code for use with `Pool.map()`. Each process will run its own Monte Carlo estimate, and the results can be combined as

$$
\pi = 4 \times \frac{\sum_{N_{procs}} Number\ of\ Points\ in\ Circle}{\sum_{N_{procs}} Total\ Number\ of\ Points}
$$

In [3]:
import numpy as np

n_samples = int(1e6) # "e" notation defaults to float

# Newer version of NumPy recommend using a Generator object
# See https://numpy.org/doc/stable/reference/random/index.html#random-quick-start
rng = np.random.default_rng() # uses current CPU clock time as seed
x = rng.uniform(0, 1, n_samples)
y = rng.uniform(0, 1, n_samples)
inside_circle = (x**2 + y**2) <= 1 # return array of True and False
n_inside_circle = np.sum(inside_circle) # count all the Trues (aka 1s)

pi_estimate = 4 * n_inside_circle/n_samples
print(f"With {n_samples} samples, pi is {pi_estimate}")

With 1000000 samples, pi is 3.14092


---
### Exercises

1. First, convert the above algorithm into a function called `sample_quarter_circle`. It should accept a number of samples as input, and return the number of samples inside the (quarter) circle. Test that your function can approximate pi.

Note: Why not return the estimate of pi? It is easier take the ratio *after* combining the counts from each Monte Carlo run.

2. `Pool.map()` requires two arguments: `func` and `iterable`. Your function `sample_quarter_circle` fulfills the `func` requirement. Each process will be given an item from `iterable` and will apply `func` to that item. For simplicity, `iterable` can be a Python list. This list should then fulfill the following requirements:

    a. Each element in the list must be a reasonable argument for `sample_quarter_circle`
    
    b. We only need to run the Monte Carlo once per process, so the length of the list should match the number of processes. 